In [ ]:
import os
os.environ['OMP_NUM_THREADS'] = '30'

from qutip import *
import numpy as np

folder_save_figure = '../Figures/'
folder_name        = '../Data/'

In [ ]:
from __future__ import division

import itertools

from cycler import cycler
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import fssa

In [ ]:
def QFI(w0):
    
    delta_w0  = w0 / div
    
    ########### no increment
    H          = w0 * J_x
    rho        = steadystate(H      , c_ops, method = 'direct', use_rcm = True, sparse = True)

    ########### plus increment
    H_plus     = (w0 + delta_w0) * J_x
    rho_plus   = steadystate(H_plus , c_ops, method = 'direct', use_rcm = True, sparse = True)

    ########### minus increment
    H_minus    = (w0 - delta_w0) * J_x
    rho_minus  = steadystate(H_minus, c_ops, method = 'direct', use_rcm = True, sparse = True)
    
    ########### 2 plus increment
    H_2plus    = (w0 + (2 * delta_w0)) * J_x
    rho_2plus  = steadystate(H_2plus , c_ops, method = 'direct', use_rcm = True, sparse = True)

    ########### 2 minus increment
    H_2minus   = (w0 - (2 * delta_w0)) * J_x
    rho_2minus = steadystate(H_2minus, c_ops, method = 'direct', use_rcm = True, sparse = True)
    
    #Derivative
    diff       = (-rho_2plus + 8*rho_plus - 8*rho_minus + rho_2minus) / (12 * delta_w0)
    
    #
    val, vec   = rho.eigenstates(sort='high')
    
    L          = 0
    for i in range( len(val) ):
        
        for j in range( len(val) ):
            
            if val[i] + val[j] >= tol:
                
                veci = vec[i].unit()
                vecj = vec[j].unit()

                den  = val[i] + val[j]

                ope  = veci * vecj.dag()

                L    = L + 2 * ( (veci.dag() * diff * vecj)[0,0] / den ) * ope
            
    Q   = np.real( ( rho * (L * L) ).tr() )
        
    return Q

N_vec1 = range(2, 20, 2)
N_vec2 = range(20, 1020, 20)

N_vec  = np.concatenate( (N_vec1, N_vec2), axis = 0 )

for N in N_vec:

    div     = 400

    J_x     = jmat( N/2, 'x')
    J_minus = jmat( N/2, '-')

    kappa   = 1
    c_ops   = [np.sqrt( kappa / (N/2) ) * J_minus]

    w0_vec1 = np.linspace(0.10, 0.9  , 50)
    w0_vec2 = np.linspace(0.90, 1.0  , 50)
    w0_vec3 = np.linspace(1.00, 1.2  , 50)

    w0_vec  = np.concatenate( (w0_vec1, w0_vec2, w0_vec3), axis = 0 )

    tol     = 1e-8

    QFI_vec = parfor(QFI, w0_vec)

    np.save(folder_name + "QFI_along_w0_N="+str(N) , QFI_vec ) 
    np.save(folder_name + "w0_N="+str(N)           , w0_vec  )

In [ ]:
#Finite-size scaling as in Figure 2 - BTC file. Using https://pyfssa.readthedocs.io/en/stable/index.html

folder_name = '../Data/'

N_vec1      = range(6, 20, 2)
N_vec2      = range(20, 520, 20)

N_vec       = np.concatenate( (N_vec1, N_vec2), axis = 0 )

st          = 35

ls          = N_vec
a           = []
w0_vec      = []

for N in N_vec:
    
    aux = np.load(folder_name + "QFI_along_w0_N="+str(N)+".npy")
    aux = aux[st:-1]
    a.append( aux )

rhos   = np.load(folder_name  + "w0_N="+str(N)+".npy")[st:-1]

da     = np.multiply(a , 0.001)
ret    = fssa.autoscale(ls, rhos, a, da, 0.99, 1.45, 2)

auto_scaled_data_1 = fssa.scaledata(ls, rhos, a, da, ret.rho, ret.nu, ret.zeta)

print("wcritical = ", ret.rho, ret.drho)
print("nu = ", ret.nu, ret.dnu)
print("zeta = ", ret.zeta, ret.dzeta)